### Build model and test follow operation

In [27]:
import os
import sys 
from pathlib import Path 

%load_ext autoreload
%autoreload 2


base_dir = Path(os.getcwd()).parents[0]
sys.path.append(str(base_dir))

from utils import data_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Triplets

In [29]:
data_dir = base_dir/'data'
kg_path = data_dir/'kb.txt'
os.listdir(data_dir)
assert kg_path.exists()

triplets, entity_to_idx, relation_to_idx, idx_to_entity, idx_to_relation = data_utils.load_triplets(kg_path)
num_entities = len(entity_to_idx)

num entities: 43234
num relations: 9
num triplets  134741


### Load question transformer

In [30]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')
model = AutoModel.from_pretrained('microsoft/MiniLM-L12-H384-uncased')

### Load dataset 

In [31]:
import warnings

qa_dir = data_dir/'1-hop-20220731T054004Z-001/1-hop/vanilla'
qa_paths = os.listdir(qa_dir)

qa_path = qa_dir/qa_paths[0]

print(qa_path)
val_pairs = data_utils.load_qa_pairs(qa_path)
data_utils.santity_check(val_pairs, entity_to_idx)

C:\Users\Sidhant\Documents\projects_research\graph_qa\kgqa\data\1-hop-20220731T054004Z-001\1-hop\vanilla\qa_dev.txt
Num qa pairs loaded: 9992
number of entities missing 0: []


### Tokenize dataset

In [32]:
# # Tokenize sentences
sentences = [row[0] for row in val_pairs]
val_tokens = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### Create Torch Dataset

In [33]:
class QADataset(Dataset):
    def __init__(self, qa_pairs, q_tokens, entity_to_idx):
        self.qa_pairs = qa_pairs
        self.entity_to_idx = entity_to_idx
        self.q_tokens = q_tokens

    def __len__(self):
        return len(self.qa_pairs)

    def __getitem__(self, idx):
        pad = 5
        token_sample = {key: self.q_tokens[key][idx, :] for key in self.q_tokens}
        
        head_entities = self.qa_pairs[idx][2]
        head_entities = [self.entity_to_idx[entity] for entity in head_entities]
                
        # padded
        head_entities = self.pad_entity(head_entities, pad=pad)
        head_entities = torch.LongTensor(head_entities)
        tail_entities = self.qa_pairs[idx][1]
        tail_entities = [self.entity_to_idx[entity] for entity in tail_entities]
        
        tail_entities = self.pad_entity(tail_entities, pad=pad)
        tail_entities = torch.LongTensor(tail_entities)
        
        return token_sample, head_entities, tail_entities
        
    
    def pad_entity(self, list1, pad=5):
        # max length should be this 
        if len(list1) > pad:
            list1 = list1[:pad]
        
        assert len(list1) > 0
        
        list1 = list1 + [-1] * (pad-len(list1))
        return list1
        
        
val_dataset = QADataset(val_pairs, val_tokens, entity_to_idx)


dl = DataLoader(val_dataset, batch_size=10)

for batch in dl:
    break
    
batch[0]['input_ids'].shape

torch.Size([10, 23])

In [35]:
## Test forward pass of transfomer 

with torch.no_grad():
    out = model(**batch[0])
    out.pooler_output

### Create differentiable KG

In [36]:
subject_matrix, rel_matrix, object_matrix = data_utils.create_differentiable_kg(triplets, entity_to_idx, relation_to_idx)
object_matrix = torch.transpose(object_matrix, 0, 1)


### Create model

In [38]:
import models_onehop

net = models_onehop.GNNLightning(model, subject_matrix, rel_matrix, object_matrix)

### Test Follow Operation

In [39]:
# this model assumes only 1 subject exists
subject_ids = batch[1][:, 0]
# print(subject_ids)
subject_vector = torch.zeros(batch[1].shape[0], num_entities)
subject_vector[range(0, subject_vector.shape[0]), subject_ids] = 1
subject_vector = torch.transpose(subject_vector, 0, 1)

In [40]:
# def create_sparse_subject():
#     bs = batch[1].shape[0]
#     bs_ids = torch.Tensor(range(0, bs))
#     indices = torch.cat([subject_ids[None, :], bs_ids[None, :]])
#     values = torch.FloatTensor([1] * len(bs_ids))
#     subject_vector = torch.sparse_coo_tensor(indices, values, size=(len(entity_to_idx), bs))
    
#     return subject_vector

# # subject_vector = create_sparse_subject()
# subject_vector = torch.transpose(subject_vector, 0, 1)

In [41]:
print(subject_vector.shape, subject_matrix.shape)
bs = batch[1].shape[0]
relation = torch.randn(bs, 9)



print('relation ', relation.shape)
print('subject_vector', subject_vector.shape)
print('subject_matrix', subject_matrix.shape)
print('rel_matrix ', rel_matrix.shape)
print('object_matrix1 ', object_matrix.shape)

out = net.follow(subject_vector, relation, subject_matrix, rel_matrix, object_matrix)
out.shape

torch.Size([43234, 10]) torch.Size([134741, 43234])
relation  torch.Size([10, 9])
subject_vector torch.Size([43234, 10])
subject_matrix torch.Size([134741, 43234])
rel_matrix  torch.Size([134741, 9])
object_matrix1  torch.Size([43234, 134741])


torch.Size([10, 43234])